In [1]:
import pandas as pd
from clickhouse_driver import Client
from unidecode import unidecode

In [2]:
client = Client(host = 'localhost',
                client_name = 'python-driver',
                port = 49000,
                user='primedata',
                password='primedata',
                secure=False,
                verify=False,
                database = 'eventify_stag',
               )

In [15]:
citi = client.query_dataframe("""select distinct(str_val) from profile_str_final_v where str_key = 'city' """)

In [25]:
city=[]
for i in citi['str_val']:
    txt= unidecode(i)
    city.append(txt)
    

In [26]:
city

['Ha Noi',
 'Da Nang',
 'Quang Binh',
 'Lai Chau',
 'Thai Binh',
 'Dong Nai',
 'Bac Ninh',
 'Lao Cai',
 'Yen Bai',
 'Quang Ngai',
 'Thanh Hoa',
 'Thai Nguyen',
 'Tuyen Quang',
 'Khanh Hoa',
 'Bac Giang',
 'Quang Ninh',
 'TP Ho Chi Minh',
 'Nghe An',
 'Hai Duong',
 'Vinh Phuc',
 'Phu Yen',
 'Lam Dong',
 'Phu Tho',
 'Ninh Binh',
 'Hai Phong',
 'Hung Yen',
 'Gia Lai',
 'Can Tho']

# view erp null màu sắc nhvt10

In [6]:
erp = pd.read_csv(r'C:\Users\ADMIN\Desktop\hello\Sale dashboard\erp\erp.txt',sep='\t',encoding='utf-8')

c:\users\admin\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
product_check = pd.read_csv(r'C:\Users\ADMIN\Desktop\product_null.csv',sep = ',',encoding='utf-8')

In [45]:
lsp_mavt_check = list(product_check.ma_vt.unique())

In [9]:
check = erp[['ma_vt','nh_vt10','t_tien_sau_ck']].copy()

In [11]:
check[check['nh_vt10'].isna()]

,ma_vt,nh_vt10,t_tien_sau_ck
634166,APS060S8KPE02B058106,NaN,225000.0
641982,APS060S8KPE02G004106,NaN,225000.0
686235,KPS018S9KPE03G008104,NaN,230000.0
695286,KSO009S9WCO03N026064,NaN,180556.0
695432,KPS002S9KPE02G088023,NaN,202500.0


In [39]:
import requests
from datetime import datetime
from datetime import timedelta

In [21]:
def daterange_api(start_date, end_date):
    for n in range(0,int((end_date - start_date).days),180):
        yield start_date + timedelta(n)

In [22]:
def get_product_new(ngay_bd,ngay_kt,headers):
    payload = {'ngay_bd': ngay_bd,'ngay_kt': ngay_kt}
    r = requests.get("https://crm.aristino.com:4436/KGAPI/api/CategoryAPI/GetProducts_ByUpdate",headers={'key': headers},
                     params = payload)
    return r

In [26]:
from pandas.io.json import json_normalize
def read_frame(data):
    df = pd.read_json(data.text)
    df = pd.io.json.json_normalize(df.Result)
    return df

In [33]:
key ='58cf21a9-af57-4fdd-8234-034e05aedfc2'
start_date = datetime(2020,1,1)
end_date = datetime(2021,12,7)


datetime.datetime(2020, 1, 1, 0, 0)

In [41]:
product =[]
for single_date in daterange_api(start_date, end_date):
    batch_start = single_date.strftime("%Y-%m-%d")
    batch_end = (single_date + timedelta(days=180)).strftime("%Y-%m-%d")
    query= get_product_new(batch_start,batch_end,key)
    result = read_frame(query)
    product.append(result)
    print(batch_start,batch_end,'status: Done \n','shape: ',result.shape)
    
    
    

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_3324/1110607934.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = pd.io.json.json_normalize(df.Result)


2020-01-01 2020-06-29 status: Done 
 shape:  (3166, 49)
2020-06-29 2020-12-26 status: Done 
 shape:  (3551, 49)
2020-12-26 2021-06-24 status: Done 
 shape:  (16780, 49)
2021-06-24 2021-12-21 status: Done 
 shape:  (23179, 49)


In [43]:
product_Newapi = pd.concat(product).reset_index(drop=True)

In [46]:
product_Newapi[product_Newapi['ma_vt'].isin(lsp_mavt_check)]

,ma_vt,ten_vt,ma_vt2,ten_vt2,dvt,nh_vt1,nh_vt2,nh_vt3,nh_vt4,nh_vt5,...,ten_nh_vt14,ten_nh_vt15,ten_nh_vt16,ten_nh_vt17,ten_nh_vt18,ten_nh_vt19,ten_nh_vt20,ten_nh_vt21,ghi_chu,status


In [80]:
nh_vt10_newAPi = product_Newapi[['ma_vt','nh_vt10','ten_nh_vt10']].copy()

In [83]:
nh_vt10_newAPi['val']=1


In [88]:
nh_vt10_newAPi=nh_vt10_newAPi.groupby(['ma_vt','nh_vt10','ten_nh_vt10']).sum().reset_index()

In [89]:
nh_vt10_newAPi

,ma_vt,nh_vt10,ten_nh_vt10,val
0,1190131400X.0,N10-NL.NONE,Không phân biệt màu NL,1
1,1190131500X.0,N10-NL.NONE,Không phân biệt màu NL,1
2,1190131600X.0,N10-NL.NONE,Không phân biệt màu NL,1
3,1190131700X.0,N10-NL.NONE,Không phân biệt màu NL,1
4,1190131800X.0,N10-NL.NONE,Không phân biệt màu NL,1
...,...,...,...,...
46608,ZTBKDS00000000000000,N10-XXXXXX,Không phân biệt màu,1
46609,ZTBKDS000000000000N0,N10-XXXXXX,Không phân biệt màu,1
46610,ZTBNIC00TPB000000000,N10-XXX,Không phân biệt màu,1
46611,ZTBSTR00TPB000000000,N10-000000,Không phân biệt màu,1


# product posgress

In [47]:
import pandas as pd
import psycopg2

In [155]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [62]:
conn = psycopg2.connect(
    host="localhost",
    user="pd",
    password="iloveprimedata",
    dbname='eventify_stag',
    port=45432)

In [51]:
cur = conn.cursor()

In [52]:
query_product = pd.read_sql_query("""select * from public.product_item;""",conn)

In [56]:
query_product

,tenant_id,id,product_id,properties
0,1,KSO001S1G06100.0,KSO001S1G06100,{'ten_vt': 'CCDC-Handloom Quần soóc Kwin KSO00...
1,1,APS049S1N03706.0,APS049S1N03706,{'ten_vt': 'CCDC-Handloom Áo thun có cổ ngắn t...
2,1,APS049S1W00406.0,APS049S1W00406,{'ten_vt': 'CCDC-Handloom Áo thun có cổ ngắn t...
3,1,31902ABS200.2,31902ABS200,{'ten_vt': 'BTP -Quần đùi Aristino ABS02 màu M...
4,1,UTR01701N00000000000,UTR01701N0000000000,{'ten_vt': 'Đồng phục quần âu nam đơn Ao Châu ...
...,...,...,...,...
79696,1,APS005S1T00607.0,APS005S1T00607,{'ten_vt': 'CCDC-Handloom Áo thun có cổ ngắn t...
79697,1,APS005S1N04107.0,APS005S1N04107,{'ten_vt': 'CCDC-Handloom Áo thun có cổ ngắn t...
79698,1,ASO019S1E06607.0,ASO019S1E06607,{'ten_vt': 'CCDC-Handloom Quần soóc Aristino A...
79699,1,KSO001S1G14600.0,KSO001S1G14600,{'ten_vt': 'CCDC-Handloom Quần soóc Kwin KSO00...


In [54]:
query_product2 = pd.io.json.json_normalize(query_product.properties)

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_3324/1497685251.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  query_product2 = pd.io.json.json_normalize(query_product.properties)


In [58]:
product_pg = pd.concat([query_product[['id','product_id']],query_product2],axis=1)

In [66]:
nh_vt10_pg=product_pg[product_pg['id'].isin(lsp_mavt_check)][['id','nh_vt10','ten_nh_vt10']]

In [69]:
import numpy as np
nh_vt10_pg.replace(np.NaN,1,inplace=True)

In [71]:
nh_vt10_pg=nh_vt10_pg.groupby(['id','nh_vt10']).agg({'ten_nh_vt10':list}).reset_index()

In [73]:
nh_vt10_pg['ten_nh_vt10n']=[list(set(i)) for i in nh_vt10_pg['ten_nh_vt10']]


In [75]:
nh_vt10_pg['ten_nh_vt10n'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[1]    4854
Name: ten_nh_vt10n, dtype: int64

In [95]:
nh_vt10_check = nh_vt10_pg.merge(nh_vt10_newAPi[['nh_vt10','ten_nh_vt10']],on='nh_vt10',how='left')

In [99]:
nh_vt10_check[nh_vt10_check['ten_nh_vt10_y'].isna()]

,id,nh_vt10,ten_nh_vt10_x,ten_nh_vt10n,ten_nh_vt10_y
0,ABL001070LT23GK01221,N10-GK0122,[1],[1],NaN
1,ABL001070LT23GK01222,N10-GK0122,[1],[1],NaN
2,ABL001070LT23GK01223,N10-GK0122,[1],[1],NaN
3,ABL002070LT23YK01221,N10-YK0122,[1],[1],NaN
4,ABL002070LT23YK01222,N10-YK0122,[1],[1],NaN
...,...,...,...,...,...
4850,KWO010W8KAC02B194003,N10-B19400,[1],[1],NaN
4851,KWO010W8KAC02B194005,N10-B19400,[1],[1],NaN
4852,KWO010W8KAC02N121004,N10-N12100,[1],[1],NaN
4853,KWO010W8KAC02R072004,N10-R07200,[1],[1],NaN


In [100]:
product_pg[product_pg['id'].isin(lsp_mavt_check)]

,id,product_id,ten_vt,ten_vt2,dvt,nh_vt1,nh_vt2,nh_vt3,nh_vt4,nh_vt5,nh_vt6,nh_vt7,nh_vt8,nh_vt9,nh_vt10,nh_vt11,nh_vt12,nh_vt13,nh_vt14,nh_vt15,nh_vt16,nh_vt17,nh_vt18,nh_vt19,nh_vt20,nh_vt21,ten_nh_vt1,ten_nh_vt2,ten_nh_vt3,ten_nh_vt4,ten_nh_vt5,ten_nh_vt6,ten_nh_vt7,ten_nh_vt8,ten_nh_vt9,ten_nh_vt10,ten_nh_vt11,ten_nh_vt12,ten_nh_vt13,ten_nh_vt14,ten_nh_vt15,ten_nh_vt16,ten_nh_vt17,ten_nh_vt18,ten_nh_vt19,ten_nh_vt20,ten_nh_vt21,ghi_chu,status
39,ASS122S7WMO03I122065,ASS122S7WMO03I12206,áo sơ mi Aristino ASS122S7 màu Trắng X cỡ 41,áo sơ mi Aristino ASS122S7 màu Trắng X,CHIEC,ASS122S7,N02-ARISTINO,N03-SOMINGANTAY-SO,N04-S,N05-2018,N06-MM,N07-SOMI,N08-SOMINGANTAY,N09-OFFICE,N10-I12206,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1
40,ASS122S7WMO03I122066,ASS122S7WMO03I12206,áo sơ mi Aristino ASS122S7 màu Trắng X cỡ 42,áo sơ mi Aristino ASS122S7 màu Trắng X,CHIEC,ASS122S7,N02-ARISTINO,N03-SOMINGANTAY-SO,N04-S,N05-2018,N06-MM,N07-SOMI,N08-SOMINGANTAY,N09-OFFICE,N10-I12206,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1
44,ASS122S8WMO03J122072,ASS122S8WMO03J12207,áo sơ mi ngắn tay Aristino ASS122S8 màu Caro đ...,áo sơ mi ngắn tay Aristino ASS122S8 màu Caro đ...,CHIEC,ASS122S8,N02-ARISTINO,N03-SOMINGANTAY-SO,N04-S,N05-2019,N06-MM,N07-SOMI,N08-SOMINGANTAY,N09-NONE,N10-J12207,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1
45,ASS122S8WMO03J122073,ASS122S8WMO03J12207,áo sơ mi ngắn tay Aristino ASS122S8 màu Caro đ...,áo sơ mi ngắn tay Aristino ASS122S8 màu Caro đ...,CHIEC,ASS122S8,N02-ARISTINO,N03-SOMINGANTAY-SO,N04-S,N05-2019,N06-MM,N07-SOMI,N08-SOMINGANTAY,N09-NONE,N10-J12207,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1
46,ASS122S8WMO03J122074,ASS122S8WMO03J12207,áo sơ mi ngắn tay Aristino ASS122S8 màu Caro đ...,áo sơ mi ngắn tay Aristino ASS122S8 màu Caro đ...,CHIEC,ASS122S8,N02-ARISTINO,N03-SOMINGANTAY-SO,N04-S,N05-2019,N06-MM,N07-SOMI,N08-SOMINGANTAY,N09-NONE,N10-J12207,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79666,ASS121S8WMO03J121073,ASS121S8WMO03J12107,áo sơ mi ngắn tay Aristino ASS121S8 màu Caro đ...,áo sơ mi ngắn tay Aristino ASS121S8 màu Caro đ...,CHIEC,ASS121S8,N02-ARISTINO,N03-SOMINGANTAY-SO,N04-S,N05-2019,N06-MM,N07-SOMI,N08-SOMINGANTAY,N09-NONE,N10-J12107,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1
79667,ASS121S8WMO03J121074,ASS121S8WMO03J12107,áo sơ mi ngắn tay Aristino ASS121S8 màu Caro đ...,áo sơ mi ngắn tay Aristino ASS121S8 màu Caro đ...,CHIEC,ASS121S8,N02-ARISTINO,N03-SOMINGANTAY-SO,N04-S,N05-2019,N06-MM,N07-SOMI,N08-SOMINGANTAY,N09-NONE,N10-J12107,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1
79668,ASS121S8WMO03J121075,ASS121S8WMO03J12107,áo sơ mi ngắn tay Aristino ASS121S8 màu Caro đ...,áo sơ mi ngắn tay Aristino ASS121S8 màu Caro đ...,CHIEC,ASS121S8,N02-ARISTINO,N03-SOMINGANTAY-SO,N04-S,N05-2019,N06-MM,N07-SOMI,N08-SOMINGANTAY,N09-NONE,N10-J12107,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1
79669,ASS121S8WMO03J121076,ASS121S8WMO03J12107,áo sơ mi ngắn tay Aristino ASS121S8 màu Caro đ...,áo sơ mi ngắn tay Aristino ASS121S8 màu Caro đ...,CHIEC,ASS121S8,N02-ARISTINO,N03-SOMINGANTAY-SO,N04-S,N05-2019,N06-MM,N07-SOMI,N08-SOMINGANTAY,N09-NONE,N10-J12107,42,

In [146]:
my_string="hello python world , i'm a beginner "
print (my_string.split("world",1)[0])

hello python 


In [149]:
def str_spilit_befor(x):
    if 'cỡ' in x:
        my_str = x.split("cỡ",1)[0]
    else:
        my_str= x
    return my_str

In [110]:
def str_spilit(x):
    my_str = x.split("màu",1)[1]
    return my_str

In [157]:
color = product_pg[product_pg['id'].isin(lsp_mavt_check)][['nh_vt10','ten_nh_vt10','id','product_id','ten_vt','ten_vt2']]

In [158]:
color.reset_index(drop=True,inplace=True)

In [159]:
color['color']=color['ten_vt'].copy()

In [160]:
colors = []
for i in color.color:
    if 'màu' in i:
        colors.append(str_spilit(i))
    else:
        colors.append('k')

In [161]:
color['color'] = colors

In [162]:
color['color_transform'] = color['color'].apply(str_spilit_befor)

In [248]:
color['color_transform'].value_counts().head(500)

k                                       359
 Xanh biển 20                            27
 Trắng 6 Slub                            24
 Đỏ 4                                    24
 Đen 10                                  22
 Xanh lá 4                               22
 Xanh biển 57                            20
 Xám 25 MF                               20
 Booc đô 21 MF                           18
 Xám 66                                  18
 Xanh tím than 44                        18
 Xanh biển 132                           18
 Nâu 14 MF                               17
 Xám 138 M                               17
 Xanh biển 33                            17
 Vàng/ Đen                               17
 Xanh Navy                               16
 Xanh tím than 52                        16
 Nâu 13 MF                               16
 Xanh tím than 92 MF                     16
 Xanh lá mạ 9 MF                         15
 Xám 50                                  15
 Trắng 6 Emboss                 

In [193]:
color_null = color[color['color_transform']=='k'].copy()

In [245]:
color_null.reset_index(drop=True,inplace=True)

In [167]:
color['product_id'][0][6:8]

'S7'

In [246]:
color_new=[]
for productId,colornew in zip (color_null.product_id,color_null.ten_vt2):
    if "-" in productId:
        color_new.append(colornew.split(productId[8:10],1)[1])
    else:
        color_new.append(colornew.split(productId[6:8],1)[1])

IndexError: list index out of range

In [181]:
color['product_id'][1488][8:10]#.split(x[8:10],1)[1]

'09'

In [189]:
color['color_new']=[T ]

ValueError: Length of values (5138) does not match length of index (4854)

In [205]:
len(color_new)

643

In [231]:
color_null

,nh_vt10,ten_nh_vt10,id,product_id,ten_vt,ten_vt2,color,color_transform
445,N10-G08300,NaN,AKK015S7WCO03G083002,AKK015S7WCO03G08300,Quần Khaki aristino AKK015S7 Xám 83 cỡ 29,Quần Khaki aristino AKK015S7 Xám 83,k,k
446,N10-G08300,NaN,AKK015S7WCO03G083003,AKK015S7WCO03G08300,Quần Khaki aristino AKK015S7 Xám 83 cỡ 30,Quần Khaki aristino AKK015S7 Xám 83,k,k
447,N10-G08300,NaN,AKK015S7WCO03G083004,AKK015S7WCO03G08300,Quần Khaki aristino AKK015S7 Xám 83 cỡ 31,Quần Khaki aristino AKK015S7 Xám 83,k,k
448,N10-G08300,NaN,AKK015S7WCO03G083005,AKK015S7WCO03G08300,Quần Khaki aristino AKK015S7 Xám 83 cỡ 32,Quần Khaki aristino AKK015S7 Xám 83,k,k
449,N10-G08300,NaN,AKK015S7WCO03G083006,AKK015S7WCO03G08300,Quần Khaki aristino AKK015S7 Xám 83 cỡ 33,Quần Khaki aristino AKK015S7 Xám 83,k,k
450,N10-G08300,NaN,AKK015S7WCO03G083007,AKK015S7WCO03G08300,Quần Khaki aristino AKK015S7 Xám 83 cỡ 34,Quần Khaki aristino AKK015S7 Xám 83,k,k
451,N10-G08300,NaN,AKK015S7WCO03G083008,AKK015S7WCO03G08300,Quần Khaki aristino AKK015S7 Xám 83 cỡ 35,Quần Khaki aristino AKK015S7 Xám 83,k,k
452,N10-G08300,NaN,AKK015S7WCO03G083009,AKK015S7WCO03G08300,Quần Khaki aristino AKK015S7 Xám 83 cỡ 36,Quần Khaki aristino AKK015S7 Xám 83,k,k
453,N10-N05200,NaN,AKK015S7WCO03N052003,AKK015S7WCO03N05200,Quần Khaki aristino AKK015S7 Xanh tím than 52 ...,Quần Khaki aristino AKK015S7 Xanh tím than 52,k,k
454,N10-N05200,NaN,AKK015S7WCO03N052004,AKK015S7WCO03N05200,Quần Khaki aristino AKK015S7 Xanh tím than 52 ...,Quần Khaki aristino AKK015S7 Xanh tím than 52,k,k


In [230]:
len(color_new)

359

In [228]:
color_null['product_id'][4255][8:10]# in color_null['ten_vt2'][1449]

'14'

In [219]:
color_null['ten_vt2'][1449].split(color_null['product_id'][1449][8:10],1)[1]

' bản trung Navy họa tiết nhí'

In [251]:
color.rename(columns={'id':"ma_vt"},inplace=True)

In [253]:
erp.merge(color[['ma_vt','color_transform']],on='ma_vt',how='left')

,anonymous_id,at,order_value,t_tien_sau_ck,so_luong,stt_rec,dich_vu,dvt,nguon_dh,status,ma_vt,ma_vt2,ma_ct_km,nh_dvcs1,nh_dvcs3,gender,str_val,card_tier_name,age,interval_age,nh_vt2,nh_vt5,nh_vt8,nh_vt10,nh_vt12,nh_vt13,nh_vt14,segment,color_transform
0,1uch0mnuiDglIgn288MqYYQV2Mc,2020-01-07 18:45:46,795000.0,795000.0,1.0,K1000000002098287BL2,NaN,NaN,NaN,Hoàn thành,ATR01208KTR02N025105,ATR01208KTR02N02510,NaN,SRMB,KQMB,NaN,undefined,undefined,NaN,undefined,N02-ARISTINO,N05-2019,N08-QUANAU,N10-N02510,NaN,NaN,NaN,other,NaN
1,1uch0mnuiDglIgn288MqYYQV2Mc,2020-01-07 18:45:46,750000.0,750000.0,1.0,K1000000002098287BL2,NaN,NaN,NaN,Hoàn thành,AWO032W7KVC02Y032003,AWO032W7KVC02Y03200,NaN,SRMB,KQMB,NaN,undefined,undefined,NaN,undefined,N02-ARISTINO,N05-2018,N08-AOLEN,N10-Y03200,NaN,NaN,NaN,other,Vàng 32
2,1uchC3Ds4YcBjDv0BqbbrpL3hgY,2020-01-05 13:12:52,625000.0,625000.0,1.0,K1000000002094349BL2,NaN,NaN,NaN,Hoàn thành,ALS31607WBA02I316075,ALS31607WBA02I31607,NaN,SRMB,KQMB,NaN,undefined,undefined,NaN,undefined,N02-ARISTINO,N05-2018,N08-SOMIDAITAY,N10-I31607,N12-07,N13-02,N14-BA,other,NaN
3,1uchC3Ds4YcBjDv0BqbbrpL3hgY,2020-01-05 13:12:52,795000.0,795000.0,1.0,K1000000002094349BL2,NaN,NaN,NaN,Hoàn thành,ALS59808WMO02J598076,ALS59808WMO02J59807,NaN,SRMB,KQMB,NaN,undefined,undefined,NaN,undefined,N02-ARISTINO,N05-2019,N08-SOMIDAITAY,N10-J59807,N12-07,N13-02,N14-MO,other,NaN
4,1uchC3Ds4YcBjDv0BqbbrpL3hgY,2020-01-05 13:12:52,40000.0,40000.0,1.0,K1000000002094349BL2,NaN,NaN,NaN,Hoàn thành,BM000-SC03BA000.0,BM000-SC03BA000,NaN,SRMB,KQMB,NaN,undefined,undefined,NaN,undefined,N02-BIZMEN,N05-NONE,N08-TATNAM,N10-000,NaN,N13-34,N14-BA,other,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871934,21vFprFfS18ZQEe98VpunT4LZBf,2021-12-06 16:02:31,725000.0,725000.0,1.0,K1000000004275937BL2,NaN,CHIEC,NaN,1,AKK00608WCO03K001004,AKK00608WCO03K00100,NaN,SRMT,DQMT,Nam,Đà Nẵng,undefined,52.0,>50,N02-ARISTINO,N05-2019,N08-QUANKHAKI,N10-K00100,N12-00,N13-03,N14-CO,big_spender,NaN
871935,21vFprFfS18ZQEe98VpunT4LZBf,2021-12-06 16:02:31,695000.0,695000.0,1.0,K1000000004275937BL2,NaN,CHIEC,NaN,1,ASS057S1WMO15M057064,ASS057S1WMO15M05706,NaN,SRMT,DQMT,Nam,Đà Nẵng,undefined,52.0,>50,N02-ARISTINO,N05-2021,N08-SOMINGANTAY,N10-M05706,N12-06,N13-15,N14-MO,big_spender,NaN
871936,21vFprFfS18ZQEe98VpunT4LZBf,2021-12-06 16:02:31,550000.0,550000.0,1.0,K1000000004275937BL2,NaN,CHIEC,NaN,1,APS027S1KPE02K010094,APS027S1KPE02K01009,NaN,SRMT,DQMT,Nam,Đà Nẵng,undefined,52.0,>50,N02-ARISTINO,N05-2021,N08-POLO,N10-K01009,N12-09,N13-02,N14-PE,big_spender,NaN
871937,21vFpuqXAMQoTpGQkvl6H91G00t,2021-12-06 09:17:45,2950000.0,2950000.0,1.0,K1000000004274924BL2,NaN,CHIEC,NaN,1,ACL00409LLT00G000220,ACL00409LLT00G00022,NaN,SRMT,DQMT,Nữ,Khánh Hòa,undefined,33.0,31-35,N02-ARISTINO,N05-2020,N08-PHUKIEN,N10-G00022,N12-22,N13-00,N14-LT,big_spender,NaN


In [254]:
erp[erp['ma_vt'].isin(lsp_mavt_check)]

,anonymous_id,at,order_value,t_tien_sau_ck,so_luong,stt_rec,dich_vu,dvt,nguon_dh,status,ma_vt,ma_vt2,ma_ct_km,nh_dvcs1,nh_dvcs3,gender,str_val,card_tier_name,age,interval_age,nh_vt2,nh_vt5,nh_vt8,nh_vt10,nh_vt12,nh_vt13,nh_vt14,segment
1,1uch0mnuiDglIgn288MqYYQV2Mc,2020-01-07 18:45:46,750000.0,750000.0,1.0,K1000000002098287BL2,NaN,NaN,NaN,Hoàn thành,AWO032W7KVC02Y032003,AWO032W7KVC02Y03200,NaN,SRMB,KQMB,NaN,undefined,undefined,NaN,undefined,N02-ARISTINO,N05-2018,N08-AOLEN,N10-Y03200,NaN,NaN,NaN,other
13,1ucyn1u45i84oxoYyuOA9WLzwqK,2020-01-06 20:00:13,750000.0,562500.0,1.0,K1000000002096850BL2,NaN,NaN,NaN,Hoàn thành,ATR00107WTR02K010003,ATR00107WTR02K01000,NaN,SRMN,DQMN,NaN,undefined,undefined,NaN,undefined,N02-ARISTINO,N05-2018,N08-QUANAU,N10-K01000,NaN,NaN,NaN,loyal_customer
17,1ud5RCicukgH5nGbXthgyoHKjWT,2020-01-06 20:00:13,750000.0,562500.0,1.0,K1000000002096850BL2,NaN,NaN,NaN,Hoàn thành,ATR00107WTR02K010003,ATR00107WTR02K01000,NaN,SRMN,DQMN,Nam,Hà Nội,SILVER,NaN,undefined,N02-ARISTINO,N05-2018,N08-QUANAU,N10-K01000,NaN,NaN,NaN,loyal_customer
24,1uceUki4fTTbWmoJFL3LB5W9lwD,2020-01-04 08:25:21,3950000.0,3950000.0,1.0,K1000000002091518BL2,NaN,NaN,NaN,Hoàn thành,ASU00808WPE03N051072,ASU00808WPE03N05107,NaN,SRMB,DQMB,NaN,undefined,undefined,NaN,undefined,N02-ARISTINO,N05-2019,N08-VEST-SUITS,N10-N05107,NaN,NaN,NaN,at_risk
43,1ucz2YU1QAA2Yn376skQsIlGqTN,2020-01-04 08:25:21,3950000.0,3950000.0,1.0,K1000000002091518BL2,NaN,NaN,NaN,Hoàn thành,ASU00808WPE03N051072,ASU00808WPE03N05107,NaN,SRMB,DQMB,Nam,Hà Nội,SILVER,NaN,undefined,N02-ARISTINO,N05-2019,N08-VEST-SUITS,N10-N05107,NaN,NaN,NaN,at_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871200,21mmG7JOoDPuNfgXWVAK3xL36O2,2021-12-03 20:47:02,3950000.0,3950000.0,1.0,K1000000004270981BL2,NaN,BO,NaN,1,ASU00108WTR03T041003,ASU00108WTR03T04100,NaN,SRMB,KQMB,Nữ,Hưng Yên,SILVER,25.0,20-25,N02-ARISTINO,N05-2019,N08-VEST-SUITS,N10-T04100,NaN,NaN,NaN,champion
871330,21sQJV3D2f3xAlu5E19FKOjH0FR,2021-12-05 19:41:33,995000.0,995000.0,1.0,K1000000004274320BL2,NaN,CHIEC,NaN,1,AKK02507WCO02E055014,AKK02507WCO02E05501,NaN,SRMB,KQMB,Nữ,Hà Nội,undefined,34.0,31-35,N02-ARISTINO,N05-2018,N08-QUANKHAKI,N10-E05501,NaN,NaN,NaN,recent_customer
871442,21sQRcD9FF5Lt353aAJS6F2XxKk,2021-12-05 17:03:56,995000.0,995000.0,1.0,K1000000004274098BL2,NaN,CHIEC,NaN,1,AKK02507WCO02N038012,AKK02507WCO02N03801,NaN,SRMB,KQMB,Nam,Hà Nội,undefined,37.0,36-40,N02-ARISTINO,N05-2018,N08-QUANKHAKI,N10-N03801,NaN,NaN,NaN,big_spender
871646,21sQl0UOBDtEvzD5cbHOgmxVNTc,2021-12-05 21:04:28,995000.0,995000.0,1.0,K1000000004274468BL2,NaN,CHIEC,NaN,1,AKK02507WCO02E055012,AKK02507WCO02E05501,NaN,SRMB,DQMB,Nam,Hà Nội,undefined,30.0,26-30,N02-ARISTINO,N05-2018,N08-QUANKHAKI,N10-E05501,NaN,NaN,NaN,big_spender


# segment

In [270]:
def create_rfm(data,end_time):
    df = data[['anonymous_id','at','t_tien_sau_ck','stt_rec']].copy()
    df['at'] = pd.to_datetime(df['at'])
    df_grouped_sum_t_tien_sau_ck = df.groupby(['anonymous_id','stt_rec','at']).sum().reset_index()
    df_calculate_rfm = df_grouped_sum_t_tien_sau_ck.groupby('anonymous_id').agg(
                        {'at': lambda x: (end_time - x.max()).days,
                        'stt_rec': lambda x: len(x),
                        't_tien_sau_ck' : lambda x : sum(x)}).reset_index()
    df_calculate_rfm.columns = ['anonymous_id','r','f','m']
    segment = transform_segment(df_calculate_rfm)
    segment['date'] = end_time
    return segment

In [268]:
def chage_str_to_time(time):
    time_changed = datetime.strptime(f"{time}", '%Y-%m-%d %H:%M:%S')
    return time_changed

In [264]:
def transform_segment(data):
    df = data.copy()
    descriptive = df.describe()
    #monetary
    min_m = descriptive['m'][3]
    m1 = descriptive['m'][4]
    m2 = descriptive['m'][5]
    m3 = descriptive['m'][6]
    max_m = descriptive['m'][7]
    #frequency
    min_f = descriptive['f'][3]
    f1 = descriptive['f'][4]
    f2 = descriptive['f'][5]
    f3 = descriptive['f'][6]
    max_f = descriptive['f'][7]
    #recency
    min_r = descriptive['r'][3]
    r1 = descriptive['r'][4]
    r2 = descriptive['r'][5]
    r3 = descriptive['r'][6]
    max_r = descriptive['r'][7]
    f_q = []
    r_q = []
    m_q = []
    for i in df.f:
        if (i >= min_f) & (i <= f1):
            i = 4
        elif (i > f1)&(i <= f2):
            i = 3
        elif (i > f2)&(i <= f3):
            i = 2
        elif (i > f3)&(i <= max_f):
            i = 1
        f_q.append(i)
    for x in df.r:
        if (x >= min_r) & (x <= r1):
            x = 1
        elif (x > r1)&(x <= r2):
            x = 2
        elif (x > r2)&(x <= r3):
            x = 3
        elif (x > r3)&(x <= max_r):
            x = 4
        r_q.append(x)
    for u in df.m:
        if (u >= min_m) & (u <= m1):
            u = 4
        elif (u > m1)&(u <= m2):
            u = 3
        elif (u > m2)&(u <= m3):
            u = 2
        elif (u > m3)&(u <= max_m):
            u = 1
        m_q.append(u)
    df['r_q'] = r_q
    df['m_q'] = m_q
    df['f_q'] = f_q
    #create rank
    rank = []
    for (i1,i2,i3) in zip(df.r_q,df.f_q,df.m_q):
        rank.append(str(i1)+str(i2)+str(i3))
    df['rank'] = rank
    #create check rank
    check = []
    for i in rank: #check rank
        if (i=='111') or (i == '311') or (i == '411') or (i == '444'):
            i = 1
        else:
            i = 0
        check.append(i)
    df['check'] = check
    #create segment: RFM
    #111: Những khách hàng đã mua gần đây nhất và thường xuyên nhất và chi tiêu nhiều nhất.
    #XX1: Những khách hàng đã chi tiêu nhiều nhất.
    #X1X: Những khách hàng đã mua thường xuyên nhất.
    #1XX: Những khách hàng đã mua hàng gần đây.
    #311: Những khách hàng đã không mua trong một thời gian, nhưng trước đây họ đã mua hàng thường xuyên và chi tiêu nhiều nhất.
    #411: Những khách hàng đã mua trong một thời gian rất dài, nhưng trước đây họ đã mua hàng thường xuyên và chi tiêu nhiều nhất.
    #444: Những khách hàng đã mua trong một thời gian rất dài, có thể đã mua một lần và chi tiêu ít nhất.
    #other: other

    segment = []
    for (i,x) in zip(df['rank'],df['check']):
        if i == '111':
            s = 'champion'
        elif i == '311':
            s = 'at_risk'
        elif i == '411':
            s = 'lost_customer'
        elif i == '444':
            s = 'lost_inconsequential_customers'
        elif (i[2] == '1') & (x == 0):
            s = 'big_spender'
        elif (i[1] == '1') & (x == 0):
            s = 'loyal_customer'
        elif (i[0] == '1') & (x == 0):
            s = 'recent_customer'
        else:
            s = 'other'
        segment.append(s)
    df['segment'] = segment
    return df


In [265]:
import time
from pandas.tseries.offsets import MonthEnd

In [271]:
segment_lsp=[]
erp_segment=[]
for beg in pd.date_range('2020-01-01 00:00:00', '2021-12-31 00:00:00', freq='MS'):
    a= time.time()
    start_time_erp=beg.strftime("%Y-%m-%d %H:%M:%S")
    start_time = datetime(2020,1,1,0,0,0).strftime("%Y-%m-%d %H:%M:%S")
    end_time = (beg + MonthEnd(1) + timedelta(seconds=59, minutes=59, hours=23)).strftime("%Y-%m-%d %H:%M:%S")
    data_segment = erp[(erp['at']>=start_time) & (erp['at']<end_time)].copy()
    segment = create_rfm(data_segment,chage_str_to_time(end_time))
    segment_lsp.append(segment)
    b=time.time()
    print('segment done',b)
    erp_merge = erp[(erp['at']>=start_time_erp) & (erp['at']<end_time)].copy()
    erp_merged = erp_merge.merge(segment[['anonymous_id','segment']],on='anonymous_id',how ='left')
    erp_segment.append(erp_merged)
    c=time.time()
    print('Time range:',start_time_erp,'-----',end_time,'\n pipeline done: ',c-a)
    
    
    
    
    

segment done 1639365163.570052
Time range: 2020-01-01 00:00:00 ----- 2020-01-31 23:59:59 
 pipeline done:  1.972109317779541
segment done 1639365166.1724992
Time range: 2020-02-01 00:00:00 ----- 2020-02-29 23:59:59 
 pipeline done:  2.567448377609253
segment done 1639365169.438689
Time range: 2020-03-01 00:00:00 ----- 2020-03-31 23:59:59 
 pipeline done:  3.2731895446777344
segment done 1639365173.1675315
Time range: 2020-04-01 00:00:00 ----- 2020-04-30 23:59:59 
 pipeline done:  3.7220888137817383
segment done 1639365178.1071322
Time range: 2020-05-01 00:00:00 ----- 2020-05-31 23:59:59 
 pipeline done:  4.989351987838745
segment done 1639365184.2183905
Time range: 2020-06-01 00:00:00 ----- 2020-06-30 23:59:59 
 pipeline done:  6.112222194671631
segment done 1639365191.6862838
Time range: 2020-07-01 00:00:00 ----- 2020-07-31 23:59:59 
 pipeline done:  7.485933780670166
segment done 1639365200.545183
Time range: 2020-08-01 00:00:00 ----- 2020-08-31 23:59:59 
 pipeline done:  8.836894512

# segment

In [275]:
client.execute('show tables')

[('attribution_views',),
 ('campaign_data_joined_v',),
 ('campaign_performance_data',),
 ('channel_metrics',),
 ('events',),
 ('events_attribution_test',),
 ('events_campaign',),
 ('events_conversion',),
 ('events_total_distinct_users_by_source_scope_ts',),
 ('events_total_distinct_users_by_source_scope_ts_mv',),
 ('kafka_events_error',),
 ('metric_result',),
 ('profile_arr',),
 ('profile_arr_final',),
 ('profile_arr_final_mv',),
 ('profile_arr_final_v',),
 ('profile_num',),
 ('profile_num_final',),
 ('profile_num_final_mv',),
 ('profile_num_final_v',),
 ('profile_str',),
 ('profile_str_final',),
 ('profile_str_final_mv',),
 ('profile_str_final_v',),
 ('profile_v',),
 ('ptrait_result',),
 ('rec_result',),
 ('rfm_segmentation_script',),
 ('rfm_segmentation_segments',),
 ('segment_funnel_temp',),
 ('segment_user',),
 ('segment_user_final',),
 ('segment_user_final_mv',),
 ('segment_user_final_v',),
 ('segment_user_historical_queue',),
 ('segment_user_preview',),
 ('segments',),
 ('segment

In [121]:
client.query_dataframe("""select *
    from segments_v limit 5""")

,tenant_id,segment_id,name,description,type,status,conditions,at_final
0,1,21jjL8H9bnhVEhxEwgJ3Wp39NS3,Phân khúc KH Dưới 25 tuổi - Tập trung 2,,historical_live,published,"[{""conditionValue"":""booleanCondition"",""type"":""...",2021-12-02 16:11:24.054
1,1,21jmaSergUXrwSJr3ALfN79aUD6,Phân khúc KH 25 -< 35 tuổi Kh ko mục tiêu,,historical_live,published,"[{""conditionValue"":""booleanCondition"",""type"":""...",2021-12-02 16:36:33.532
2,1,1xk9B0StGSwmaMkMY9rAFTdCKId,CRM_KH Không thuộc phân khúc ( age <25),"R >150, F 2-4 lần, M 1-<5 triệu, tuổi dưới 25",historical_live,paused,"[{""conditionValue"":""booleanCondition"",""type"":""...",2021-09-06 00:37:16.705
3,1,1xl4RTQM8rS44U9sebNccLlpwdi,CRM_Y Khách hàng mua mua 1 lần và hạng Sơ...,Khách hàng mua hàng 1 lần trên ERP\nHiện ...,historical_live,paused,"[{""conditionValue"":""booleanCondition"",""type"":""...",2021-11-10 07:42:56.857
4,1,1xko2PJQIsktVALSO9VUf7Y1rbf,CRM_ KH tập trung ít ( Age >50) 1630908239,"R >120 ≤ 150, F ≥2<4, M≥1<5tr, Age >50",historical_live,paused,"[{""conditionValue"":""booleanCondition"",""type"":""...",2021-11-10 07:42:56.272


In [22]:
client = Client(host = 'localhost',
                client_name = 'python-driver',
                port = 19000,
                user='primedata',
                password='primedata',
                secure=False,
                verify=False,
                database = 'eventify',
               )

# Query all segment 

In [3]:
import json
from datetime import datetime
import time

In [28]:
client = Client(host = 'localhost',
                client_name = 'python-driver',
                port = 19000,
                user='primedata',
                password='primedata',
                secure=False,
                verify=False,
                database = 'eventify',
               )

In [5]:
query_segment ="""select user,arrayDistinct(groupArray(name)) as segment_id from
    (
    select user,segment_id
    from segment_user_final_v
    ) a  
left outer join
    (
    select segment_id ,name 
    from segments_v
    ) b  
on a.segment_id = b.segment_id 
group by user """

In [6]:
client.query_dataframe('select count(*) from segment_user_final_v')

,count__
0,6213707


In [8]:
client.query_dataframe('select count(distinct(user)) from segment_user_final_v')

,uniqExact_user_
0,1406274


In [29]:
query="""select user,groupArray(segment_id) as segment_id from segment_user_final_v group by user"""

In [34]:
#test
query=client.query_dataframe("""
select * from
(select anonymous_id 
from events 
where event_name='erp_item_order_completed') a
left outer join
(select user,groupArray(segment_id) as segment_id 
from segment_user_final_v 
group by user) b
on a.anonymous_id = b.user
limit 10 
""")

,anonymous_id,user,segment_id
0,1ucfB2mfqwDX7K7FtxYRUZVZyjW,1ucfB2mfqwDX7K7FtxYRUZVZyjW,"[1ugxfvOXVMizR7dEG1mFVIa0Y4Z, 1vyKVV4IqtSn1esI..."
1,1uczdOZaO3fq0XQjn7SXAiYN63d,1uczdOZaO3fq0XQjn7SXAiYN63d,"[1v0pyrwkrhMrATPcrN7tx8wmEuF, 1xGGKqwMXiTg52Qt..."
2,1uczdOZaO3fq0XQjn7SXAiYN63d,1uczdOZaO3fq0XQjn7SXAiYN63d,"[1v0pyrwkrhMrATPcrN7tx8wmEuF, 1xGGKqwMXiTg52Qt..."
3,1udTFlyMZog1kE3TeZE0N9Uy66L,1udTFlyMZog1kE3TeZE0N9Uy66L,"[delete_right_now, 1xGBH9jJhM5fsL7RLlJp6p1p2uY..."
4,1udTFlyMZog1kE3TeZE0N9Uy66L,1udTFlyMZog1kE3TeZE0N9Uy66L,"[delete_right_now, 1xGBH9jJhM5fsL7RLlJp6p1p2uY..."
5,1udTGg7vymR358A9GGEZoufQBRk,1udTGg7vymR358A9GGEZoufQBRk,"[1xGGKqwMXiTg52Qt4PLhllOrl7c, 1yTW3l2CU6bmhJ2A..."
6,1udTGg7vymR358A9GGEZoufQBRk,1udTGg7vymR358A9GGEZoufQBRk,"[1xGGKqwMXiTg52Qt4PLhllOrl7c, 1yTW3l2CU6bmhJ2A..."
7,1udTKjxIn903DW1Ym0SwaunWbOB,1udTKjxIn903DW1Ym0SwaunWbOB,"[1xmwp50tuS2GiEeueXnQFXoXTVv, 1xGBsdkZVRjgZHIs..."
8,1udTKjxIn903DW1Ym0SwaunWbOB,1udTKjxIn903DW1Ym0SwaunWbOB,"[1xmwp50tuS2GiEeueXnQFXoXTVv, 1xGBsdkZVRjgZHIs..."
9,1udTKjxIn903DW1Ym0SwaunWbOB,1udTKjxIn903DW1Ym0SwaunWbOB,"[1xmwp50tuS2GiEeueXnQFXoXTVv, 1xGBsdkZVRjgZHIs..."


In [30]:
iterator= client.execute_iter(query, {'max_block_size': 5000},with_column_types=True)

In [31]:
columns = next(iterator)

ServerException: Code: 62.
DB::Exception: Syntax error: failed at position 82 ('a') (line 4, col 46): a
left outer join
(select user,groupArray(segment_id) as segment_id 
from segment_user_final_v 
group by user) b
on a.anonymous_id = b.user
. Expected one of: UNION, end of query, INTO OUTFILE, SETTINGS, FORMAT. Stack trace:

0. DB::parseQueryAndMovePosition(DB::IParser&, char const*&, char const*, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, bool, unsigned long, unsigned long) @ 0x115e68a6 in /usr/bin/clickhouse
1. ? @ 0xeebc79e in /usr/bin/clickhouse
2. DB::executeQuery(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, DB::Context&, bool, DB::QueryProcessingStage::Enum, bool) @ 0xeebc3d3 in /usr/bin/clickhouse
3. DB::TCPHandler::runImpl() @ 0xf64d2dd in /usr/bin/clickhouse
4. DB::TCPHandler::run() @ 0xf65f879 in /usr/bin/clickhouse
5. Poco::Net::TCPServerConnection::start() @ 0x11d138af in /u

In [9]:
columns

[('user', 'String'), ('segment_id', 'Array(String)')]

In [10]:
data_UserAndSegment=[]
for item in iterator:
    data_UserAndSegment.append(item)

In [11]:
len(data_UserAndSegment)

1533250

In [15]:
df_segmentanduser=pd.DataFrame(data_UserAndSegment,columns = [col[0] for col in columns])

In [16]:
df_segmentanduser = df_segmentanduser.explode('segment_id')

In [14]:
df_segmentanduser=df_segmentanduser.drop(index=[0,1,2]).reset_index(drop=True)

In [17]:
df_segmentanduser

,user,segment_id
0,224f9Fh98V4auE50HdD8zQPAS2d,2302cIjDyZu1x5eidCaAivSkVRU
0,224f9Fh98V4auE50HdD8zQPAS2d,22kaMN9vBKdo4WmdFmaCZ2x5dPN
0,224f9Fh98V4auE50HdD8zQPAS2d,1xAZ5X7hjs8ViNbsf61hM4wWkkm
0,224f9Fh98V4auE50HdD8zQPAS2d,2302IuGTEbQem67UB6FRjnXxRRk
0,224f9Fh98V4auE50HdD8zQPAS2d,21wxq6OEL5d61i3vDpfGO7UG1n8
...,...,...
1533249,1vegvjP38kagTXCk4dLkrhJISSO,21wxq6OEL5d61i3vDpfGO7UG1n8
1533249,1vegvjP38kagTXCk4dLkrhJISSO,21ivhIK3dHsvR8A2HcIrTRbvQp8
1533249,1vegvjP38kagTXCk4dLkrhJISSO,2302IuGTEbQem67UB6FRjnXxRRk
1533249,1vegvjP38kagTXCk4dLkrhJISSO,1xAZ5X7hjs8ViNbsf61hM4wWkkm


In [113]:
df_segmentanduser.to_csv('segment_cdp',mode = 'w', sep = '\t', header = True, index = None,encoding = 'utf-8')

In [18]:
user=df_segmentanduser[['user']].copy()
user['val'] = 1
user=user.groupby('user').sum().reset_index()

In [19]:
segment_name = client.query_dataframe("""select segment_id,name,at_final from segments_v""")

In [27]:
segment_name.to_csv('segment_name.txt',mode = 'w', sep = '\t', header = True, index = None,encoding = 'utf-8')

In [ ]:
def transform_and_jsondump(tableName,data_push):
    if tableName=='erp' or tableName=='fact_segment':
        data_push.drop(columns= 'at',inplace=True)
    else:
        data_push-data_push
    for col in data_push:
        if data_push[col].dtypes == 'datetime64[ns]':
            data_push[col] = data_push[col].astype(str)
    data_push = data_push.to_dict('records')
    data_push = json.dumps(data_push)
    return data_push

# thu gọn segment

In [20]:
segment_name['segment_transform'] = [f"a{i+1}" for i in range(len(segment_name))] 

In [21]:
df_segment_new = df_segmentanduser.merge(segment_name[['segment_id','segment_transform']],on = 'segment_id',how='left')

In [22]:
df_segment_new

,user,segment_id,segment_transform
0,224f9Fh98V4auE50HdD8zQPAS2d,22kaMN9vBKdo4WmdFmaCZ2x5dPN,a81
1,224f9Fh98V4auE50HdD8zQPAS2d,21wxq6OEL5d61i3vDpfGO7UG1n8,a90
2,224f9Fh98V4auE50HdD8zQPAS2d,21ivhIK3dHsvR8A2HcIrTRbvQp8,a58
3,21PSz3Vuj6JIdaYnhXgaNe7y8Ls,21wxq6OEL5d61i3vDpfGO7UG1n8,a90
4,21PSz3Vuj6JIdaYnhXgaNe7y8Ls,21ivhIK3dHsvR8A2HcIrTRbvQp8,a58
...,...,...,...
6556677,21vIPhHmMX7SkNSW9TQn5cL5cUB,22kaMN9vBKdo4WmdFmaCZ2x5dPN,a81
6556678,21vIPhHmMX7SkNSW9TQn5cL5cUB,21ivhIK3dHsvR8A2HcIrTRbvQp8,a58
6556679,21vIPhHmMX7SkNSW9TQn5cL5cUB,21wxq6OEL5d61i3vDpfGO7UG1n8,a90
6556680,1vegvjP38kagTXCk4dLkrhJISSO,21wxq6OEL5d61i3vDpfGO7UG1n8,a90


In [22]:
df_segment_new.drop(columns='segment_id',inplace=True)

In [23]:
df_segment_new.to_csv('segment_cdp_new',mode = 'w', sep = '\t', header = True, index = None,encoding = 'utf-8')

In [31]:
user.drop(columns='val',inplace =True)

In [28]:
segment_name

,segment_id,name,at_final,segment_transform
0,21aWsRmwKeZIDsJq5pTLnFctyky,KH mua 1lần từ 31 - 45 tuổi - SRMN,2021-12-28 21:24:39.905,a1
1,20i6CDsnJlNCSCHleOgrFcWxQFg,SR_Giang_KH mua từ 1/1/21->31/10/21,2021-11-10 07:42:54.690,a2
2,21jjL8H9bnhVEhxEwgJ3Wp39NS3,Phân khúc KH Dưới 25 tuổi - Tập trung 2,2021-12-28 21:00:15.135,a3
3,21jmaSergUXrwSJr3ALfN79aUD6,Phân khúc KH 25 -< 35 tuổi Kh ko mục tiêu,2021-12-28 21:00:22.940,a4
4,22flSLJjQQZk9kpeWG6Ox9xYoWc,HCM Customers,2021-12-28 21:21:52.316,a5
...,...,...,...,...
97,21fZr4kzZyOPUCqbRUWt07VrBEG,KH phát sinh mua hàng năm 2021 mua >=8 lần SRMT,2021-12-28 21:21:50.200,a98
98,20iu2pV3BCM3z0JXSRKfEheTrci,Survey_Nga_Khách hàng đã từng mua hàng SO MI ...,2021-11-22 21:45:43.433,a99
99,20iAOmSq6oDQxGwUAdTUIUN4hUk,SR_Giang_Tần suất 9 lần_KH mua từ 1/1/21->31/1...,2021-11-10 07:42:55.399,a100
100,21Gj3QAcTbUmf5Dm4WLCPAnTGGV,KH MB 2020-22/11/2021 1637574118,2021-12-28 21:33:07.212,a101


In [24]:
user=df_segmentanduser['user'].unique()

In [25]:
user = pd.DataFrame(data=user,columns=['user'])

In [47]:
user.drop_duplicates(inplace=True)

In [26]:
user.to_csv(r'C:\Users\ADMIN\Desktop\hello\Sale dashboard\user_id\user_id',mode = 'w', sep = '\t', header = True, index = None,encoding = 'utf-8')

In [52]:
user[user['user'].str.startswith('1smkU')]

,user
993627,1smkUFi92dAqaA39Z30tRNXaX68
